google earth enginge attempt to download clay, sand, silt layers from isric for angola

In [9]:
import ee
import time

# ------------------------------
# 1) Authenticate and initialize
# ------------------------------
ee.Authenticate()
ee.Initialize(project='soilsangolathesis-471709')

# ------------------------------
# 2) Define bounding box (EPSG:4326)
# ------------------------------
lon_min = 11.668473
lat_min = -18.042082
lon_max = 24.082117
lat_max = -4.372591
aoi = ee.Geometry.Rectangle([lon_min, lat_min, lon_max, lat_max])

# ------------------------------
# 3) Soil properties and depths
# ------------------------------
properties = ["sand", "silt", "clay", "phh2o"]
depths = ["0-5cm", "5-15cm", "15-30cm"]

# ------------------------------
# 4) Export to Google Drive
# ------------------------------
tasks = []

for prop in properties:
    img = ee.Image(f"projects/soilgrids-isric/{prop}_mean")
    for depth in depths:
        band_name = f"{prop}_{depth}_mean"  # must match exact GEE band name
        layer = img.select(band_name).clip(aoi)
        task = ee.batch.Export.image.toDrive(
            image=layer,
            description=f"{prop}_{depth}",
            folder="SoilGrids",          # Drive folder
            fileNamePrefix=f"{prop}_{depth}",
            region=aoi,
            scale=250,                    # SoilGrids native resolution
            crs="EPSG:4326",
            maxPixels=1e13
        )
        task.start()
        tasks.append(task)
        print(f"Started export: {prop} {depth}")

# ------------------------------
# 5) Monitor task status
# ------------------------------
print("\nMonitoring tasks... (this may take a while)")
while any(t.status()['state'] in ['READY', 'RUNNING'] for t in tasks):
    for t in tasks:
        status = t.status()
        print(f"{status['description']}: {status['state']}")
    time.sleep(30)  # wait 30s before checking again

print("\nAll tasks completed or failed. Check your Google Drive folder 'SoilGrids'.")


Started export: sand 0-5cm
Started export: sand 5-15cm
Started export: sand 15-30cm
Started export: silt 0-5cm
Started export: silt 5-15cm
Started export: silt 15-30cm
Started export: clay 0-5cm
Started export: clay 5-15cm
Started export: clay 15-30cm
Started export: phh2o 0-5cm
Started export: phh2o 5-15cm
Started export: phh2o 15-30cm

Monitoring tasks... (this may take a while)
sand_0-5cm: READY
sand_5-15cm: READY
sand_15-30cm: READY
silt_0-5cm: READY
silt_5-15cm: READY
silt_15-30cm: READY
clay_0-5cm: READY
clay_5-15cm: READY
clay_15-30cm: READY
phh2o_0-5cm: READY
phh2o_5-15cm: READY
phh2o_15-30cm: READY
sand_0-5cm: RUNNING
sand_5-15cm: RUNNING
sand_15-30cm: READY
silt_0-5cm: READY
silt_5-15cm: READY
silt_15-30cm: READY
clay_0-5cm: READY
clay_5-15cm: READY
clay_15-30cm: READY
phh2o_0-5cm: READY
phh2o_5-15cm: READY
phh2o_15-30cm: READY
sand_0-5cm: RUNNING
sand_5-15cm: RUNNING
sand_15-30cm: READY
silt_0-5cm: READY
silt_5-15cm: READY
silt_15-30cm: READY
clay_0-5cm: READY
clay_5-15cm: R

In [8]:
import ee
ee.Initialize(project='soilsangolathesis-471709')

tasks = ee.batch.Task.list()
for t in tasks:
    print(t.id, t.status())


QHBEQCCA2QSXUKAEVH7LB4YT {'state': 'FAILED', 'description': 'phh2o_15-30cm', 'priority': 100, 'creation_timestamp_ms': 1757499778954, 'update_timestamp_ms': 1757499848801, 'start_timestamp_ms': 1757499848240, 'task_type': 'EXPORT_IMAGE', 'attempt': 1, 'error_message': "Image.select: Band pattern 'sl3' did not match any bands. Available bands: [phh2o_0-5cm_mean, phh2o_5-15cm_mean, phh2o_15-30cm_mean, phh2o_30-60cm_mean, phh2o_60-100cm_mean, phh2o_100-200cm_mean]", 'id': 'QHBEQCCA2QSXUKAEVH7LB4YT', 'name': 'projects/soilsangolathesis-471709/operations/QHBEQCCA2QSXUKAEVH7LB4YT'}
GCWWRCEC5QBFHYOECSZGH5ER {'state': 'FAILED', 'description': 'phh2o_5-15cm', 'priority': 100, 'creation_timestamp_ms': 1757499778035, 'update_timestamp_ms': 1757499838809, 'start_timestamp_ms': 1757499838235, 'task_type': 'EXPORT_IMAGE', 'attempt': 1, 'error_message': "Image.select: Band pattern 'sl2' did not match any bands. Available bands: [phh2o_0-5cm_mean, phh2o_5-15cm_mean, phh2o_15-30cm_mean, phh2o_30-60cm_m